In [3]:
import os
import sys
import cv2
import pytesseract
from PIL import Image, ImageDraw,ImageFont
from pytesseract import Output

In [4]:
nb_dir = '/'.join(os.getcwd().split('/')[:-1])
sys.path.append(nb_dir)
sys.path.append(os.path.split(nb_dir)[0])

import config
import src.utilities.app_context as app_context
app_context.init()

#path to craft model, weight can be pulled form the production bracnh of repo
config.CRAFT_MODEL_PATH= nb_dir + '/utilities/craft_pytorch/model/craft_mlt_25k.pth'
config.CRAFT_REFINE_MODEL_PATH = nb_dir + '/utilities/craft_pytorch/model/craft_refiner_CTW1500.pth'


In [6]:
from src.services.main import TextDetection

In [4]:
#base_dir = '/home/dhiraj/Documents/Anuwad/anuvaad/anuvaad-etl/anuvaad-extractor/block-merger/src/notebooks/sample-data/input'
base_dir = '/home/naresh/anuvaad/anuvaad-etl/anuvaad-extractor/document-processor/word-detector/craft/upload/'
filename    = 'good.pdf'

#filename    = 'hamlet_1.pdf'
file_format = 'pdf'
language    =  'hi'

In [5]:
app_context.application_context = {
  "input":{
  "inputs": [
    {
      "file": {
        "identifier": "string",
        "name": filename,
        "type": file_format
      },
      "config": {
        "OCR": {
          "option": "HIGH_ACCURACY",
          "language": "hi"
        }
      }
    }
  ]}
}


In [6]:
resp = TextDetection(app_context,base_dir)

[2021-05-03 16:03:06,672] {loghandler.py:48} MainThread DEBUG in loghandler: Block merger starting processing {'input': {'inputs': [{'file': {'identifier': 'string', 'name': 'good.pdf', 'type': 'pdf'}, 'config': {'OCR': {'option': 'HIGH_ACCURACY', 'language': 'hi'}}}]}}
[2021-05-03 16:03:06,676] {loghandler.py:20} MainThread INFO in loghandler: created processing directories successfully /home/naresh/anuvaad/anuvaad-etl/anuvaad-extractor/document-processor/word-detector/craft/upload/good_5e128711-b37d-4d8c-a48a-57d84df72b53
[2021-05-03 16:03:06,678] {loghandler.py:20} MainThread INFO in loghandler: filepath /home/naresh/anuvaad/anuvaad-etl/anuvaad-extractor/document-processor/word-detector/craft/upload/good.pdf, working_dir /home/naresh/anuvaad/anuvaad-etl/anuvaad-extractor/document-processor/word-detector/craft/upload/good_5e128711-b37d-4d8c-a48a-57d84df72b53
[2021-05-03 16:03:07,086] {loghandler.py:20} MainThread INFO in loghandler: Extracting images of /home/naresh/anuvaad/anuvaad-e

In [ ]:
import json
for k in range(2,5):
    config.LANGUAGE_LINK_THRESOLDS['en']['link_threshold']=k*0.2
    for i in range(3):
        config.LANGUAGE_LINE_THRESOLDS['en']['low_text'] = i*0.2
        for j in range(3):
            config.LANGUAGE_LINE_THRESOLDS['en']['text_threshold'] = j*0.2

            resp = TextDetection(app_context,base_dir)
            file = "/home/naresh/word_detector/"+str(k*0.2)+"_"+str(i*0.2)+"_"+str(j*0.2)+"_.json" 
            json_object = json.dumps(resp, indent = 4)
            with open(file, "w") as outfile: 
                outfile.write(json_object) 
            #json.dump(resp, out_file) 
            #print(resp['rsp']['outputs'][0]['page_info'][0])
            image = draw_box(resp,resp['rsp']['outputs'][file_index]['page_info'][page_index],save_dir ,str(k*0.2)+"_"+str(i*0.2)+"_"+str(j*0.2),color="green", save=True)  


In [ ]:
file_index = 0
page_index =0
filepath = resp['rsp']['outputs'][file_index]['page_info'][page_index]

save_dir = '/home/naresh/word_detector_benchmark/'
#resp['rsp']['outputs'][file_index]['pages'][page_index]['words'][0]

In [ ]:

def draw_box(resp,filepath,save_dir, thresh,color="green", save=False):
    image  = Image.open(filepath)
    draw   = ImageDraw.Draw(image)
    for i in resp['rsp']['outputs'][file_index]['pages'][page_index]['words']:
        #font = ImageFont.truetype("sans-serif.ttf", 30)
        draw.text((10, 10),thresh,(255,0,255))
        print("kkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkk",i)
        draw.rectangle(((i['boundingBox']['vertices'][0]['x'], i['boundingBox']['vertices'][0]['y']), (i['boundingBox']['vertices'][2]['x'],i['boundingBox']['vertices'][2]['y'])), outline=color,width=3)
        
    
    save_filepath = os.path.join(save_dir, "bbox_"+str(thresh)+os.path.basename(filepath))
    if save:
        image.save(save_filepath)
    
    return image

In [ ]:
#local draw box
def draw_box(resp,filepath,save_dir,color="green", save=False):
    image  = Image.open(filepath)
    draw   = ImageDraw.Draw(image)
    for i in resp['rsp']['outputs'][file_index]['pages'][page_index]['words']:
        draw.rectangle(((i['boundingBox']['vertices'][0]['x'], i['boundingBox']['vertices'][0]['y']), (i['boundingBox']['vertices'][2]['x'],i['boundingBox']['vertices'][2]['y'])), outline=color,width=3)
        
    
    save_filepath = os.path.join(save_dir, "bbox_"+os.path.basename(filepath))
    if save:
        image.save(save_filepath)
    
    return image

In [ ]:
image =draw_box(resp,filepath,save_dir,color="green", save=True)       

In [ ]:
def draw_tess(filepath,save_dir):
    img = cv2.imread(filepath)
    name = filepath.split("/")[-1]
    print(name)
    h, w, c = img.shape
    boxes = pytesseract.image_to_boxes(img)
    #try:
    d = pytesseract.image_to_data(img, output_type=Output.DICT)
    n_boxes = len(d['text'])
    for i in range(n_boxes):
        if int(d['conf'][i]) > 1:
            (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
            img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imwrite(save_dir+name, img)
    print(save_dir+name)
#     except:
#         pass
    

In [ ]:
draw_tess(filepath,save_dir)